<a href="https://colab.research.google.com/github/felixchiuman/Stock-Market-Analysis/blob/main/stock_market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#library

!pip install python-telegram-bot yfinance pandas_ta nest_asyncio mplfinance nltk GoogleNews

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 10.3 MB/s eta 0:00:00


Bot Code

In [ ]:
import logging
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import nest_asyncio
import asyncio
import os
import time
import datetime
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from telegram import Update
from telegram.constants import ParseMode
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler
from google.colab import userdata
from GoogleNews import GoogleNews

# --- 1. CONFIGURATION ---
nest_asyncio.apply()

try:
    BOT_TOKEN = userdata.get('botToken')
except:
    BOT_TOKEN = "PASTE_YOUR_TOKEN_HERE_IF_USERDATA_FAILS"

nltk.download('vader_lexicon', quiet=True)
vader = SentimentIntensityAnalyzer()

MEME_URLS = {
    "BUY": "https://i.imgflip.com/1ur9b0.jpg",
    "SELL": "https://i.imgflip.com/261o3j.jpg",
    "WAIT": "https://i.imgflip.com/26am.jpg",
    "BLOCKED": "https://i.imgflip.com/4t0m5.jpg" # Panic/Stop meme
}

# --- 2. GOOGLE NEWS ENGINE ---
def get_news_sentiment(ticker_symbol):
    try:
        googlenews = GoogleNews(lang='en', region='US')
        googlenews.search(f"{ticker_symbol} stock news")
        results = googlenews.result()

        if not results: return 0.0, "⚪ Neutral", []

        total_score = 0; count = 0; headlines = []
        seen = set()

        for item in results:
            title = item.get('title', '')
            if title in seen or not title: continue
            seen.add(title)

            score = vader.polarity_scores(title)['compound']
            total_score += score
            count += 1
            headlines.append(f"• {title[:60]}...")
            if count >= 5: break

        if count == 0: return 0.0, "⚪ Neutral", []

        avg = total_score / count
        if avg > 0.15: txt = f"🟢 BULLISH ({avg:.2f})"
        elif avg < -0.15: txt = f"🔴 BEARISH ({avg:.2f})"
        else: txt = f"⚪ NEUTRAL ({avg:.2f})"

        return avg, txt, headlines
    except: return 0.0, "⚪ Error", []

# --- 3. DATA FETCHER ---
def get_chunked_data(ticker, years_back=6):
    print(f"🔄 Deep Scan: {ticker}...")
    frames = []
    curr_yr = datetime.date.today().year

    for i in range(years_back):
        y = curr_yr - i
        try:
            # Standard yf call (No session)
            stock = yf.Ticker(ticker)
            # Handle current year vs past years
            end = datetime.date.today().strftime("%Y-%m-%d") if y == curr_yr else f"{y}-12-31"
            df = stock.history(start=f"{y}-01-01", end=end)

            if not df.empty: frames.append(df)
            time.sleep(0.3)
        except: pass

    if not frames: return pd.DataFrame()
    return pd.concat(frames).sort_index()[~pd.concat(frames).index.duplicated(keep='first')]

# --- 4. BACKTEST ENGINE ---
def calculate_backtest_metrics(df):
    wins=0; trades=0
    closes = df['Close'].values; means = df['Mean'].values
    stds = df['StdDev'].values; z_scores = df['Z_Score'].values
    highs = df['High'].values; lows = df['Low'].values

    for i in range(len(df) - 15):
        if z_scores[i] < -2.0:
            trades += 1
            entry = closes[i]; target = means[i]; stop = entry - stds[i]
            for j in range(1, 11):
                if lows[i+j] <= stop: break # Loss
                elif highs[i+j] >= target: wins+=1; break # Win

    win_rate = (wins/trades*100) if trades > 0 else 0
    return win_rate, trades

# --- 5. HYBRID ANALYSIS ENGINE ---
def analyze_stock(ticker_input):
    ticker = ticker_input.upper()
    df = get_chunked_data(ticker)

    if df.empty and len(ticker) == 4 and ticker.isalpha():
        ticker = f"{ticker}.JK"
        df = get_chunked_data(ticker)

    if df.empty: return None, None

    curr_sym = "$"
    try:
        if yf.Ticker(ticker).info.get('currency') == 'IDR': curr_sym = "Rp"
    except: pass

    try:
        # A. NEWS & MATH
        news_score, news_text, headlines = get_news_sentiment(ticker)

        df['Mean'] = df['Close'].rolling(20).mean()
        df['StdDev'] = df['Close'].rolling(20).std()
        df['Z_Score'] = (df['Close'] - df['Mean']) / df['StdDev']

        real_wr, trades = calculate_backtest_metrics(df)

        curr = df.iloc[-1]
        price = curr['Close']; z = curr['Z_Score']
        mean = curr['Mean']; std = curr['StdDev']

        # --- B. DECISION LOGIC ---

        # 1. QUANT DECISION (Pure Math)
        if z < -2.0:
            if real_wr >= 50: quant_dec = "✅ ENTER (Stat Buy)"
            else: quant_dec = "⚠️ RISKY (Low Win Rate)"
        elif z > 2.0: quant_dec = "🔴 SELL (Stat Top)"
        else: quant_dec = "💤 WAIT (No Setup)"

        # 2. HYBRID DECISION (News Filter)
        hybrid_dec = quant_dec # Default to math
        meme_key = "WAIT"

        # Apply Filter
        if ("ENTER" in quant_dec or "RISKY" in quant_dec):
            if news_score < -0.15:
                hybrid_dec = "🛑 BLOCKED (News Bearish)"
                meme_key = "BLOCKED"
            else:
                meme_key = "BUY"
        elif "SELL" in quant_dec:
            meme_key = "SELL"

        # Investor Logic
        if z < -2: inv_dec = "💎 STRONG BUY"
        elif z < -1: inv_dec = "✅ BUY"
        elif z > 2: inv_dec = "🛑 PAUSE"
        else: inv_dec = "⚪ HOLD"

        # Formatting
        def fmt(v): return f"{int(v):,}" if curr_sym == "Rp" else f"{v:,.2f}"
        head_txt = "\n".join(headlines[:3]) if headlines else "No news found."

        report = f"""
📌 **ASSET:** {ticker} (Dual Decision)
💰 **PRICE:** {curr_sym} {fmt(price)}
📊 **Z-SCORE:** {z:.2f} σ
📰 **NEWS:** {news_text}
──────────────────────
🐇 **SWING TRADER DASHBOARD**
*Horizon: 1-2 Weeks*

🧮 **MATH SIGNAL:** {quant_dec}
*(Based on {real_wr:.1f}% Win Rate)*

🧠 **HYBRID SIGNAL:** {hybrid_dec}
*(Math + News Filter)*

🎯 **Target:** {curr_sym} {fmt(mean)}
🛑 **Stop:** {curr_sym} {fmt(price-std)}
──────────────────────
🐢 **INVESTOR DASHBOARD**
*Horizon: 2+ Years*

💎 **Value Signal:** {inv_dec}
📉 **Discount:** {((mean-price)/mean)*100:.1f}% vs Fair Value
🏷️ **Fair Value:** {curr_sym} {fmt(mean)}
──────────────────────
🗞️ **TOP HEADLINES**
{head_txt}
"""
        return report, MEME_URLS[meme_key]

    except Exception as e:
        print(e)
        return None, None

# --- 6. HANDLERS ---
async def analyze(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not context.args: return await update.message.reply_text("Usage: /analyze AAPL")
    ticker = context.args[0]
    await update.message.reply_text(f"🔍 Checking Math & News for {ticker}...")

    report, meme = analyze_stock(ticker)
    if report:
        await update.message.reply_photo(meme, caption=report, parse_mode=ParseMode.MARKDOWN)
    else:
        await update.message.reply_text("❌ Error. Try again.")

async def start(update, context):
    await update.message.reply_text("🤖 Dual-Decision Bot Ready!")

async def main():
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("analyze", analyze))
    print("Polling...")
    await app.run_polling()

if __name__ == '__main__':
    try: asyncio.run(main())
    except KeyboardInterrupt: pass

Polling...
🔄 Deep Scan: MSFT...
